In [4]:
#e
import threading
import queue
import os
import math
import itertools
import copy

import datasets as hfds
import torch
import torchvision.transforms.functional as TF
import torchvision.io as TFIO

import minai.sampler as mins
import minai.datasets as minds

In [5]:
#e
def simple_collate_func(array_of_results):
    xs = [r[0] for results in array_of_results for r in results]
    ys = [r[1] for results in array_of_results for r in results]
    return xs, ys

class HFCollate:
    def __init__(self, ds: hfds.Dataset, device=None):
        self.features = tuple(ds.features.keys())
        self.device = device

    def __call__(self, array_of_results):
        collated = [[] for _ in range(len(self.features))]
        for result in array_of_results:
            for i, feature in enumerate(self.features):
                collated[i].extend(result[feature])

        for i in range(len(collated)):
            if type(collated[i][0]) is not torch.Tensor:
                collated[i] = torch.tensor(collated[i]).to(self.device)
            else:
                collated[i] = torch.stack(collated[i]).to(self.device)

        return collated
    
    def __repr__(self):
        return f"HFCollate(features={self.features}, device={self.device})"

In [6]:
#e
def clamp(a, x, b):
    return min(max(a, x), b)

In [30]:
#e
WORK_TYPE_NONE = 0
WORK_TYPE_LOAD_BATCHES = 1
WORK_TYPE_GET_ITEMS = 2
WORK_TYPE_SHUTDOWN = 100

class WorkItem:
    def __init__(self, type):
        self.type = type

    #def __del__(self): print("work item dead", self.type, threading.get_native_id())

    def __repr__(self): return f"{self.__class__.__name__}({str(vars(self))})"

class WorkItemLoadBatches(WorkItem):
    def __init__(self, cached_iter: "Collator.CachedIter", num_batches):
        super().__init__(WORK_TYPE_LOAD_BATCHES)
        self.cached_iter = cached_iter
        self.num_batches = num_batches

    #def __del__(self): print("work item loadbatches dead", threading.get_native_id())

class WorkItemGetItems(WorkItem):
    def __init__(self, group: "WorkGroup", indices):
        super().__init__(WORK_TYPE_GET_ITEMS)
        self.group = group
        self.indices = indices
        self.results = None

        self.group.gi_work_array.append(self)

    #def __del__(self): print("work item getitems dead", threading.get_native_id())

class COPTS: # CollatorOpts
    def __init__(self,
                 *, 
                 num_threads = None, 
                 debug_out_queue: queue.SimpleQueue = None):
        self.num_threads = num_threads
        self.debug_out_queue = debug_out_queue
    
    def __repr__(self): return str(vars(self))

    def finalize(self):
        cpu_count = os.cpu_count()
        if self.num_threads is None: self.num_threads = max(1, cpu_count//2)
        self.num_threads = clamp(0, self.num_threads, cpu_count*10)

        return self

class COIOPTS: # Collator(Cached)IterOpts
    DEFAULT_DEVICE = None

    def __init__(self,
                 *,
                 sampler_iter: mins.SamplerIter = None,
                 cached_batch_count = 1,
                 sub_batch_size = None,
                 getitem_func = None,
                 collate_func = None,
                 collate_device = None):

        self.sampler_iter = sampler_iter
        self.cached_batch_count = cached_batch_count
        self.sub_batch_size = sub_batch_size
        self.getitem_func = getitem_func
        self.collate_func = collate_func
        self.collate_device = collate_device or self.DEFAULT_DEVICE

    def __repr__(self): return str(vars(self))

    def finalize(self, collator_opts: COPTS):
        if collator_opts.num_threads == 0:
            self.cached_batch_count = 0
            self.sub_batch_size = self.sampler_iter.opts.batch_size
        else:
            if self.sub_batch_size is None:
                self.sub_batch_size = math.ceil(self.sampler_iter.opts.batch_size / collator_opts.num_threads)
            elif self.sub_batch_size == 0:
                self.sub_batch_size = self.sampler_iter.opts.batch_size

        self.sub_batch_size = clamp(1, self.sub_batch_size, self.sampler_iter.opts.batch_size)

        return self


class Collator:
    class CachedIter:
        def __init__(self, serial, collator_iter_opts: COIOPTS):
            self.serial = serial
            self.opts = collator_iter_opts
            self.iter = iter(collator_iter_opts.sampler_iter)
            self.num_groups_total = collator_iter_opts.sampler_iter.num_batches
            self.num_groups_requested = 0
            self.num_groups_started = 0
            self.num_groups_finished = 0
            self.lock = threading.Lock()
            self.collated_queue = queue.SimpleQueue()

        #def __del__(self): print("cached iter dead", threading.get_native_id())

        def __repr__(self):
            return f"CachedIter(serial={self.serial}, total={self.num_groups_total}, requested={self.num_groups_requested}, started={self.num_groups_started}, finished={self.num_groups_finished}, opts={self.opts})"

    def __init__(self, copts: COPTS):
        self.opts = copts.finalize()

        self.threads_spawned = False
        self.work_queue = queue.SimpleQueue()
        self.current_iter_serial = 0
        self.shutdown_counter = None
        self.registered_dl_count = 0

    #def __del__(self): print("collator dead", threading.get_native_id())

    def __repr__(self): return str(vars(self))

    def start_new_iter(self, collator_iter_opts: COIOPTS):
        if not self.threads_spawned: 
            self.shutdown_counter = threading.Semaphore(self.opts.num_threads)
            for i in range(self.opts.num_threads): 
                threading.Thread(target=collator_threadproc, args=(i+1, self)).start()
                self.shutdown_counter.acquire()
            
            self.threads_spawned = True

        self.current_iter_serial += 1
        new_iter_serial = self.current_iter_serial
        if self.opts.debug_out_queue: self.opts.debug_out_queue.put(f"[0] New iter {new_iter_serial}")

        cached_iter = self.CachedIter(new_iter_serial, collator_iter_opts)
        
        return cached_iter
    
    def load_batches(self, cached_iter: CachedIter, num_batches):
        if not num_batches: return
        
        cached_iter.lock.acquire()
        to_request = min(num_batches, cached_iter.num_groups_total - cached_iter.num_groups_requested)
        cached_iter.num_groups_requested += to_request
        cached_iter.lock.release()

        if to_request:
            self.work_queue.put(WorkItemLoadBatches(cached_iter, to_request))
            if self.opts.debug_out_queue: self.opts.debug_out_queue.put(f"[0] Requesting {to_request} batches for iter {cached_iter.serial}")

            if self.opts.num_threads == 0:
                collator_threadproc(0, self)

    def register_dl(self): 
        self.registered_dl_count += 1
    def unregister_dl(self): 
        self.registered_dl_count -= 1
        if self.registered_dl_count == 0:
            self.shutdown()

    def shutdown(self):
        if self.threads_spawned:
            for _ in range(self.opts.num_threads): self.work_queue.put(WorkItem(WORK_TYPE_SHUTDOWN))
            
            def wait_for_threads_to_shutdown_and_empty_work_queue(num_threads, shutdown_counter, work_queue):
                for _ in range(num_threads): shutdown_counter.acquire()

                while not work_queue.empty(): 
                    got = work_queue.get()
                    if type(got) is WorkItemGetItems:
                        gi_work: WorkItemGetItems = got
                        if gi_work.group: gi_work.group.complete()
            
            threading.Thread(target=wait_for_threads_to_shutdown_and_empty_work_queue, 
                             args=(self.opts.num_threads, self.shutdown_counter, self.work_queue), daemon=True).start()


class WorkGroup:
    def __init__(self, cached_iter: "Collator.CachedIter", batch_serial, num_total):
        self.cached_iter = cached_iter
        self.batch_serial = batch_serial
        self.num_total = num_total
        self.gi_work_array: list[WorkItemGetItems] = []

        self.num_done_lock = threading.Lock()
        self.num_done = 0

    #def __del__(self): print("work group dead", threading.get_native_id())

    def __repr__(self): return f"{self.__class__.__name__}(iter_serial={self.cached_iter.serial}, batch_serial={self.batch_serial}, "\
                                    f"num_done={self.num_done}, num_total={self.num_total})"

    def complete(self):
        for gi_work in self.gi_work_array: 
            gi_work.group = None

        self.gi_work_array.clear()

class CollatedResult:
    def __init__(self, batch_serial, result):
        self.batch_serial = batch_serial
        self.result = result

    #def __del__(self): print("collatedresult dead")

    def __repr__(self): return f"{self.__class__.__name__}({str(vars(self))})"


def collator_process_one_work(thread_id, work_queue, shutdown_counter, debug_out_queue):
    work: WorkItem = work_queue.get()

    if debug_out_queue: debug_out_queue.put(f"[{thread_id}] Got {work}")
    work_type = work.type
    
    if work_type == WORK_TYPE_LOAD_BATCHES:
        lb_work: WorkItemLoadBatches = work
        
        cached_iter = lb_work.cached_iter
        
        cached_iter.lock.acquire()
        read_num_groups = cached_iter.num_groups_started
        
        cached_iter.num_groups_started += lb_work.num_batches
        array_of_batch_indices = list(itertools.islice(cached_iter.iter, lb_work.num_batches))
        cached_iter.lock.release()

        batches_spawned = 0
        for batch_indices in array_of_batch_indices:
            sub_batches = mins.chunkify(batch_indices, cached_iter.opts.sub_batch_size)
            assert len(sub_batches)

            batch_serial = read_num_groups + batches_spawned
            work_group = WorkGroup(cached_iter, batch_serial + 1, len(sub_batches))
            for sub_batch in sub_batches:
                work_queue.put(WorkItemGetItems(work_group, sub_batch))
                if debug_out_queue: debug_out_queue.put(f"[{thread_id}] Queued {work_group.gi_work_array[-1]}")

            batches_spawned += 1


    elif work_type == WORK_TYPE_GET_ITEMS:
        gi_work: WorkItemGetItems = work
        work_group = gi_work.group
        cached_iter = work_group.cached_iter

        gi_work.results = cached_iter.opts.getitem_func(gi_work.indices)

        work_group.num_done_lock.acquire()
        work_group.num_done += 1
        work_group.num_done_lock.release()
        assert work_group.num_done <= work_group.num_total
        
        if work_group.num_done == work_group.num_total:
            assert work_group.num_done == len(work_group.gi_work_array)
            if debug_out_queue: debug_out_queue.put(f"[{thread_id}] Completed group {work_group}")

            
            cached_iter.lock.acquire()
            cached_iter.num_groups_finished += 1
            is_last_group = cached_iter.num_groups_finished == cached_iter.num_groups_total
            cached_iter.lock.release()

            collated = cached_iter.opts.collate_func([gi_work.results for gi_work in work_group.gi_work_array])
            cached_iter.collated_queue.put(CollatedResult(work_group.batch_serial, collated))

            if is_last_group:
                if debug_out_queue: debug_out_queue.put(f"[{thread_id}] Completed iter {cached_iter.serial}")
                cached_iter.collated_queue.put(CollatedResult(0, None))

            work_group.complete()

            if thread_id == 0:
                return True


    elif work_type == WORK_TYPE_SHUTDOWN:
        shutdown_counter.release()
        return True

    else: assert False

    return False

def collator_threadproc(thread_id: int, ctx: Collator):
    work_queue = ctx.work_queue
    shutdown_counter = ctx.shutdown_counter
    debug_out_queue = ctx.opts.debug_out_queue
    
    if debug_out_queue: debug_out_queue.put(f"[{thread_id}] Started")

    while 1:
        should_exit = collator_process_one_work(thread_id, work_queue, shutdown_counter, debug_out_queue)
        if should_exit:
            break

    if debug_out_queue: debug_out_queue.put(f"[{thread_id}] Exiting")


class DataLoaderIter:
    def __init__(self, collator: Collator, cached_iter: Collator.CachedIter):
        self.collator = collator
        self.cached_iter = cached_iter
        
        self.buffer: list[CollatedResult] = []
        self.next_batch_index = 1

    #def __del__(self): print("dliter dead", threading.get_native_id())
    
    def __repr__(self):
        return f"DataLoaderIter({self.cached_iter})"

    def __iter__(self):
        self.collator.load_batches(self.cached_iter, self.cached_iter.opts.cached_batch_count + 1)

        while 1:
            collated: CollatedResult = self.cached_iter.collated_queue.get()
            if not collated.batch_serial:
                assert not self.buffer # batch_serial == 0 should strictly come last
                break

            self.buffer.append(collated)

            while 1:
                found_i = next((i for i, x in enumerate(self.buffer) if x.batch_serial == self.next_batch_index), -1)
                if found_i == -1:
                    break

                to_yield = self.buffer[found_i].result
                self.buffer[found_i] = self.buffer[-1]
                self.buffer.pop()
                self.next_batch_index += 1
                
                yield to_yield
                self.collator.load_batches(self.cached_iter, 1)

class DataLoader:
    def __init__(self, ds, collator: Collator, collator_iter_opts: COIOPTS):
        self.ds = ds
        self.collator = collator
        self.opts = collator_iter_opts
        
        self.opts.finalize(self.collator.opts)
        self.collator.register_dl()

    def __del__(self): 
        #print("dl dead", threading.get_native_id())
        self.collator.unregister_dl()
    
    def __repr__(self): return str(vars(self))

    def __iter__(self):
        cached_iter = self.collator.start_new_iter(self.opts)
        return iter(DataLoaderIter(self.collator, cached_iter))
    
    @classmethod
    def solo(cls, ds, collator_opts: COPTS, collator_iter_opts: COIOPTS):
        collator = Collator(collator_opts)
        return cls(ds, collator, collator_iter_opts)

    @classmethod
    def simple(cls, simple_ds: minds.SimpleDataset, sampler_iter_opts: mins.SIO = None, collator_opts: COPTS = None, collator_iter_opts: COIOPTS = None):
        sampler_iter_opts = sampler_iter_opts or mins.SIO()
        collator_opts = collator_opts or COPTS()
        collator_iter_opts = collator_iter_opts or COIOPTS()

        collator_iter_opts.sampler_iter = mins.Sampler(len(simple_ds)).iter(sampler_iter_opts)
        collator_iter_opts.getitem_func = simple_ds.__getitem__
        collator_iter_opts.collate_func = simple_collate_func
        return cls.solo(simple_ds, collator_opts, collator_iter_opts)

    @classmethod
    def hf(cls, hf_ds: hfds.Dataset, sampler_iter_opts: mins.SIO = None, collator_opts: COPTS = None, collator_iter_opts: COIOPTS = None):
        assert type(hf_ds) is hfds.Dataset, f"Dataset expected, not {type(hf_ds).__name__}"
        sampler_iter_opts = sampler_iter_opts or mins.SIO()
        collator_opts = collator_opts or COPTS()
        collator_iter_opts = collator_iter_opts or COIOPTS()
        
        collator_iter_opts.sampler_iter = mins.Sampler(len(hf_ds)).iter(sampler_iter_opts)
        collator_iter_opts.getitem_func = hf_ds.__getitem__
        collator_iter_opts.collate_func = HFCollate(hf_ds, collator_iter_opts.collate_device).__call__
        return cls.solo(hf_ds, collator_opts, collator_iter_opts)
    
    @classmethod
    def hf_dsd(cls, hf_ds: hfds.Dataset, collator: Collator, sampler_iter_opts: mins.SIO = None, collator_iter_opts: COIOPTS = None):
        assert type(hf_ds) is hfds.Dataset, f"Dataset expected, not {type(hf_ds).__name__}"
        sampler_iter_opts = sampler_iter_opts or mins.SIO()
        collator_iter_opts = collator_iter_opts or COIOPTS()
        
        collator_iter_opts.sampler_iter = mins.Sampler(len(hf_ds)).iter(sampler_iter_opts)
        collator_iter_opts.getitem_func = hf_ds.__getitem__
        collator_iter_opts.collate_func = HFCollate(hf_ds, collator_iter_opts.collate_device).__call__
        return cls(hf_ds, collator, collator_iter_opts)

In [31]:
import time
import random

def getitem_func(indices):
    #time.sleep(random.random())
    return [[i, -i] for i in indices]

dl = DataLoader.solo(None, 
                     COPTS(num_threads=4),
                     COIOPTS(sampler_iter=mins.Sampler(11).iter(mins.SIO(5)), 
                             getitem_func=getitem_func, 
                             collate_func=simple_collate_func))

it1 = iter(dl)
it2 = iter(dl)
print(1, next(it1))
print(2, next(it2))
print(1, next(it1))
print(1, next(it1))
print(2, next(it2))
print(2, next(it2))
print()

for xb, yb in dl:
    print(xb, yb)
print("agane")
for xb, yb in dl:
    print(xb, yb)

1 ([0, 1, 2, 3, 4], [0, -1, -2, -3, -4])
2 ([0, 1, 2, 3, 4], [0, -1, -2, -3, -4])
1 ([5, 6, 7, 8, 9], [-5, -6, -7, -8, -9])
1 ([10, 4, 9, 1, 2], [-10, -4, -9, -1, -2])
2 ([5, 6, 7, 8, 9], [-5, -6, -7, -8, -9])
2 ([10, 0, 3, 9, 4], [-10, 0, -3, -9, -4])

[0, 1, 2, 3, 4] [0, -1, -2, -3, -4]
[5, 6, 7, 8, 9] [-5, -6, -7, -8, -9]
[10, 6, 4, 5, 4] [-10, -6, -4, -5, -4]
agane
[0, 1, 2, 3, 4] [0, -1, -2, -3, -4]
[5, 6, 7, 8, 9] [-5, -6, -7, -8, -9]
[10, 1, 7, 6, 8] [-10, -1, -7, -6, -8]


In [32]:
import gc

def foo():
    dl = DataLoader.solo(None, 
                     COPTS(num_threads=4),
                     COIOPTS(sampler_iter=mins.Sampler(11).iter(mins.SIO(5)), 
                             getitem_func=getitem_func, 
                             collate_func=simple_collate_func))
    next(iter(dl))

foo()

In [33]:
ds = minds.SimpleDataset(list(range(100)), list(range(0, -100, -1)))
dl = DataLoader.simple(ds, mins.SIO(16))
ds, vars(dl.collator), vars(dl.collator.opts)

(SimpleDataset(len=100, xs=int, ys=int),
 {'opts': {'num_threads': 7, 'debug_out_queue': None},
  'threads_spawned': False,
  'work_queue': <_queue.SimpleQueue at 0x7f808ecac270>,
  'current_iter_serial': 0,
  'shutdown_counter': None,
  'registered_dl_count': 1},
 {'num_threads': 7, 'debug_out_queue': None})

In [34]:
it = iter(dl)
print(next(it))
print(next(it))

it = iter(dl)
print(next(it))
print(next(it))

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [0, -1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15])
([16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], [-16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31])
([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [0, -1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15])
([16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], [-16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31])


In [35]:
for _ in range(2):
    for xs, ys in dl:
        print(xs, ys)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] [0, -1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15]
[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] [-16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]
[32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47] [-32, -33, -34, -35, -36, -37, -38, -39, -40, -41, -42, -43, -44, -45, -46, -47]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63] [-48, -49, -50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63]
[64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79] [-64, -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77, -78, -79]
[80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95] [-80, -81, -82, -83, -84, -85, -86, -87, -88, -89, -90, -91, -92, -93, -94, -95]
[96, 97, 98, 99, 18, 31, 26, 13, 34, 21, 96, 45, 35, 2, 59, 4] [-96, -97, -98, -99, -18, -31, -26, -13, -34, -21, -96, -45, -35, -2, -59, -4]
[0,

In [36]:
dsd = minds.hf_load(minds.HF_DATASETS.TINY_IMAGENET)
dst = dsd["train"]

Found cached dataset parquet (/home/nblzv/.cache/huggingface/datasets/zh-plus___parquet/Maysee--tiny-imagenet-2eb6c3acd8ebc62a/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/2 [00:00<?, ?it/s]

In [37]:
#e
class HFTransform:
    def __init__(self, features, transform, **extra_args):
        assert type(features) is hfds.features.features.Features
        self.features = tuple(features)
        self.transform = transform

        for k, v in extra_args.items():
            super().__setattr__(k ,v)

    def __call__(self, results):
        return self.transform(self, results)
    
    def __repr__(self):
        return f"HFTransform(features={list(self.features)})"

    @classmethod
    def ff_img_to_tensor(cls, features, n_channels=3): # first_feature
        def tf(ctx: HFTransform, results):
            xs = results[ctx.features[0]]
            for i in range(len(xs)):
                decoded = TF.to_tensor(xs[i])
                xs[i] = decoded.expand(ctx.n_channels, *decoded.shape[1:])
            return results
        
        return cls(features, tf, n_channels=n_channels)
    
    @classmethod
    def ff_img_decode_to_tensor(cls, features, n_channels=3, half=False): # first_feature
        def tf(ctx: HFTransform, results):
            xs = results[ctx.features[0]]
            for i in range(len(xs)):
                raw = torch.frombuffer(xs[i]["bytes"], dtype=torch.uint8)
                decoded = TFIO.decode_image(raw)
                decoded = decoded.expand(ctx.n_channels, *decoded.shape[1:])
                if ctx.half: decoded = decoded.half()
                else: decoded = decoded.float()
                xs[i] = decoded / 255.0
            return results
        
        return cls(features, tf, n_channels=n_channels, half=half)

In [38]:
#e
def first(iterable):
    return next(iter(iterable))

def first_value(iterable):
    return next(iter(iterable.values()))

In [39]:
def to_tensor_transform(ctx, results):
    xs = results[ctx.features[0]]
    for i in range(len(xs)):
        xs[i] = TF.to_tensor(xs[i])
    return results

thds = dst.with_transform(HFTransform(dst.features, to_tensor_transform))
dl = DataLoader.hf(thds, mins.SIO(5, False))
first(dl)[0][0].shape

torch.Size([3, 64, 64])

In [41]:
tdst = dst.with_transform(HFTransform.ff_img_to_tensor(dst.features))
def bench(cached_batch_count, next_count):
    dl = DataLoader.hf(tdst, 
                       mins.SIO(1024), 
                       collator_opts=COPTS(),
                       collator_iter_opts=COIOPTS(cached_batch_count=cached_batch_count, sub_batch_size=512))
    it = iter(dl)
    for _ in range(next_count): next(it)

print("prefetch 0")
%timeit -r 3 -n 1 bench(0, 1);
%timeit -r 3 -n 1 bench(0, 3);
print("prefetch 1")
%timeit -r 3 -n 1 bench(1, 1);
%timeit -r 3 -n 1 bench(1, 3);

prefetch 0
308 ms ± 5.38 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
852 ms ± 14.9 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
prefetch 1
1.38 s ± 117 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
2.79 s ± 122 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [42]:
tdst = dst.with_transform(HFTransform.ff_img_decode_to_tensor(dst.features)).cast_column("image", hfds.Image(decode=False))
def bench(cached_batch_count, next_count):
    dl = DataLoader.hf(tdst, mins.SIO(1024), 
                       collator_opts=COPTS(),
                       collator_iter_opts=COIOPTS(cached_batch_count=cached_batch_count, sub_batch_size=512))
    it = iter(dl)
    for _ in range(next_count): next(it)

print("prefetch 0")
%timeit -r 3 -n 1 bench(0, 1);
%timeit -r 3 -n 1 bench(0, 3);
print("prefetch 1")
%timeit -r 3 -n 1 bench(1, 1);
%timeit -r 3 -n 1 bench(1, 3);

prefetch 0


/tmp/ipykernel_19341/4268657344.py:33: UserWarning: The given buffer is not writable, and PyTorch does not support non-writable tensors. This means you can write to the underlying (supposedly non-writable) buffer using the tensor. You may want to copy the buffer to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /opt/conda/conda-bld/pytorch_1687158866840/work/torch/csrc/utils/tensor_new.cpp:1505.)
  raw = torch.frombuffer(xs[i]["bytes"], dtype=torch.uint8)


121 ms ± 13.6 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
339 ms ± 31 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
prefetch 1
645 ms ± 40.4 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
1.35 s ± 30.5 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [48]:
#e
class DataLoaderDict(dict):
    def __init__(self, collator: Collator, dataloaders_dict: dict[str, DataLoader]):
        self.collator = collator
        super().__init__(dataloaders_dict)

    def __getitem__(self, key) -> DataLoader:
        return super().__getitem__(key)
        
    def __repr__(self):
        return f"DataLoaders({list(self.keys())})"

    @classmethod
    def hf(cls, dsd: hfds.DatasetDict, 
           collator_opts: COPTS = None, 
           sampler_iter_opts: dict[str, mins.SIO] = {}, 
           collator_iter_opts: dict[str, COIOPTS] = {}):
        
        collator_opts = collator_opts or COPTS()
        collator = Collator(collator_opts)

        if type(sampler_iter_opts) is mins.SIO:
            sio_default = sampler_iter_opts
            sampler_iter_opts = {}
        else:
            sio_default = mins.SIO()

        if type(collator_iter_opts) is COIOPTS:
            coiopts_default = collator_iter_opts
            collator_iter_opts = {}
        else:
            coiopts_default = COIOPTS()

        dls = {}
        for k in dsd:
            sio = sampler_iter_opts.get(k, copy.copy(sio_default))
            coiopts = collator_iter_opts.get(k, copy.copy(coiopts_default))

            if k == "train":
                sio.shuffle = True
            else: 
                sio.batch_size *= 2

            dls[k] = DataLoader.hf_dsd(dsd[k], collator, sio, coiopts)

        return cls(collator, dls)

In [49]:
dls = DataLoaderDict.hf(dsd, COPTS(num_threads=4))
dls, {k: dls[k].opts for k in dls}

(DataLoaders(['train', 'valid']),
 {'train': {'sampler_iter': <minai.sampler.SamplerIter object at 0x7f805c418810>, 'cached_batch_count': 1, 'sub_batch_size': 16, 'getitem_func': <bound method Dataset.__getitem__ of Dataset({
      features: ['image', 'label'],
      num_rows: 100000
  })>, 'collate_func': <bound method HFCollate.__call__ of HFCollate(features=('image', 'label'), device=None)>, 'collate_device': None},
  'valid': {'sampler_iter': <minai.sampler.SamplerIter object at 0x7f808c507510>, 'cached_batch_count': 1, 'sub_batch_size': 32, 'getitem_func': <bound method Dataset.__getitem__ of Dataset({
      features: ['image', 'label'],
      num_rows: 10000
  })>, 'collate_func': <bound method HFCollate.__call__ of HFCollate(features=('image', 'label'), device=None)>, 'collate_device': None}})

In [51]:
import z_export
z_export.export()

Processing minai_nbs/datasets.ipynb -> minai/minai/datasets.py  |  same contents, skipping, took 0.001s
Processing minai_nbs/sampler.ipynb -> minai/minai/sampler.py  |  same contents, skipping, took 0.001s
Processing minai_nbs/setup+template.py -> minai/setup.py  |  same contents, skipping, took 0.000s
Processing minai_nbs/__init__+template.py -> minai/minai/__init__.py  |  same contents, skipping, took 0.000s
Processing minai_nbs/plot.ipynb -> minai/minai/plot.py  |  same contents, skipping, took 0.000s
Processing minai_nbs/mintils.py -> minai/minai/mintils.py  |  same contents, skipping, took 0.000s
Processing minai_nbs/data.ipynb -> minai/minai/data.py  |  same contents, skipping, took 0.000s

All done... took 0.004s
  lib_name: minai
  author: nblzv
  version: 0.1.1
